In [1]:
#Installing stanza package
!pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 31.0 MB/s eta 0:00:00


In [1]:
#Installing transformers for stanza package
!pip install nltk stanza transformers

In [2]:
#Importing all necessary packages
import stanza
stanza.download('en')
nlp = stanza.Pipeline()
import re
import nltk
import pandas as pd
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
nltk.download('wordnet')
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Pa

In [3]:
#Loading train dataset
train = pd.read_csv('train.csv')

In [4]:
#Displaying first 5 rows of train data
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
#Identifying rows and column in the train data
train.shape

(31962, 3)

In [6]:
#loading the test data
test = pd.read_csv('/content/test.csv')


In [7]:
#displaying the first five rows of test data
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [8]:
#identifying the rows and columns of the test data
test.shape

(17197, 2)

In [9]:
#dropping unnecesary columns
data=train.drop('id',axis=1)

In [10]:
data.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [11]:
#initialyzing the lemmatizer and stopwards
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [12]:
#cleaning the data which consists of stopwards,regular expressions and lammatizing them
def clean_text(tweet):
    tweets = " ".join(filter(lambda x: x[0]!= '@' , tweet.split()))
    tweets = re.sub('[^a-zA-Z]', ' ', tweets)
    tweets = tweets.lower()
    tweets = tweets.split()
    tweets = [word for word in tweets if not word in set(stopwords.words('english'))]
    tweets = [lemma.lemmatize(word) for word in tweets]
    tweets = " ".join(tweets)
    return tweets

In [13]:
#applying the function to the data
data['clean_tweet_text'] = data.tweet.apply(clean_text)

In [14]:
test['clean_tweet_text'] = test.tweet.apply(clean_text)

In [15]:
data.head()

,label,tweet,clean_tweet_text
0,0,@user when a father is dysfunctional and is s...,father dysfunctional selfish drag kid dysfunct...
1,0,@user @user thanks for #lyft credit i can't us...,thanks lyft credit use cause offer wheelchair ...
2,0,bihday your majesty,bihday majesty
3,0,#model i love u take with u all the time in ...,model love u take u time ur
4,0,factsguide: society now #motivation,factsguide society motivation


In [16]:
#extracting the hashtag from the data
def extract_hashtag(tweet):
    tweets = " ".join(filter(lambda x: x[0]== '#', tweet.split()))
    tweets = re.sub('[^a-zA-Z]',' ',  tweets)
    tweets = tweets.lower()
    tweets = [lemma.lemmatize(word) for word in tweets]
    tweets = "".join(tweets)
    return tweets

In [17]:
#applying the functions to  the data
data['word_with_hashtag'] = data.tweet.apply(extract_hashtag)

In [18]:
test['word_with_hashtag'] = test.tweet.apply(extract_hashtag)

In [19]:
data.head()

,label,tweet,clean_tweet_text,word_with_hashtag
0,0,@user when a father is dysfunctional and is s...,father dysfunctional selfish drag kid dysfunct...,run
1,0,@user @user thanks for #lyft credit i can't us...,thanks lyft credit use cause offer wheelchair ...,lyft disapointed getthanked
2,0,bihday your majesty,bihday majesty,
3,0,#model i love u take with u all the time in ...,model love u take u time ur,model
4,0,factsguide: society now #motivation,factsguide society motivation,motivation


In [20]:
#dropping the unnecessary columns
data=data.drop(['tweet','word_with_hashtag'],axis=1)

In [21]:
data.head()

,label,clean_tweet_text
0,0,father dysfunctional selfish drag kid dysfunct...
1,0,thanks lyft credit use cause offer wheelchair ...
2,0,bihday majesty
3,0,model love u take u time ur
4,0,factsguide society motivation


### Identifying the sentiment and sentiment score using NLTK - Sentiment **module**

In [22]:
analyzer = SentimentIntensityAnalyzer()

In [23]:
tweets = data['clean_tweet_text'][:30]

In [24]:
for i, tweet in enumerate(tweets):
    # Calculate sentiment scores for the tweet
    scores = analyzer.polarity_scores(tweet)

    # Extract compound sentiment score
    compound_score = scores['compound']

    # Determine sentiment label based on the compound score
    if compound_score > 0.05:
        sentiment_label = 'Positive'
    elif compound_score < -0.05:
        sentiment_label = 'Negative'
    else:
        sentiment_label = 'Neutral'

    # Print the results
    print(f'Tweet {i+1}:')
    print('Text:', tweet)
    print('Compound Score:', compound_score)
    print('Sentiment Label:', sentiment_label)
    print()

Tweet 1:
Text: father dysfunctional selfish drag kid dysfunction run
Compound Score: -0.7783
Sentiment Label: Negative

Tweet 2:
Text: thanks lyft credit use cause offer wheelchair van pdx disapointed getthanked
Compound Score: 0.6705
Sentiment Label: Positive

Tweet 3:
Text: bihday majesty
Compound Score: 0.0
Sentiment Label: Neutral

Tweet 4:
Text: model love u take u time ur
Compound Score: 0.6369
Sentiment Label: Positive

Tweet 5:
Text: factsguide society motivation
Compound Score: 0.34
Sentiment Label: Positive

Tweet 6:
Text: huge fan fare big talking leave chaos pay dispute get allshowandnogo
Compound Score: -0.5267
Sentiment Label: Negative

Tweet 7:
Text: camping tomorrow danny
Compound Score: 0.0
Sentiment Label: Neutral

Tweet 8:
Text: next school year year exam think school exam hate imagine actorslife revolutionschool girl
Compound Score: -0.5719
Sentiment Label: Negative

Tweet 9:
Text: love land allin cavs champion cleveland clevelandcavaliers
Compound Score: 0.8442
Sen

In [25]:
X=data['clean_tweet_text']
y=data['label']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

In [28]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

In [29]:
X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred = naive_bayes_classifier.predict(X_test_tfidf)

In [30]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the model is', accuracy)
# Print classification report
print(classification_report(y_test, y_pred))

# Print confusion matrix
print(confusion_matrix(y_test, y_pred))

Accuracy of the model is 0.9404035664007508
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5937
           1       1.00      0.16      0.28       456

    accuracy                           0.94      6393
   macro avg       0.97      0.58      0.63      6393
weighted avg       0.94      0.94      0.92      6393

[[5937    0]
 [ 381   75]]


In [31]:
test.head()

,id,tweet,clean_tweet_text,word_with_hashtag
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...,studiolife aislife requires passion dedic...
1,31964,@user #white #supremacists want everyone to s...,white supremacist want everyone see new bird m...,white supremacists birds movie
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe way heal acne altwaystoheal healthy healing,acne altwaystoheal healthy healing
3,31966,is the hp and the cursed child book up for res...,hp cursed child book reservation already yes h...,harrypotter pottermore favorite
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd bihday amazing hilarious nephew eli ahmir u...,bihday nephew


In [32]:
X_test = test['clean_tweet_text']

In [37]:
test=test.drop(['tweet','word_with_hashtag'],axis=1)

In [33]:
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [34]:
y_pred = naive_bayes_classifier.predict(X_test_tfidf)

In [35]:
test['predicted_labels'] = y_pred

In [38]:
test.head()

,id,clean_tweet_text,predicted_labels
0,31963,studiolife aislife requires passion dedication...,0
1,31964,white supremacist want everyone see new bird m...,0
2,31965,safe way heal acne altwaystoheal healthy healing,0
3,31966,hp cursed child book reservation already yes h...,0
4,31967,rd bihday amazing hilarious nephew eli ahmir u...,0


## Identifying the sentiment using stanza and sentiment score using **Roberta**

In [ ]:
#Intializing the model and its parameters
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming 3 sentiment classes  # 3 labels: negative, neutral, positive

In [ ]:
#Predicting the sentiment and sentiment scores for the first 30 tweets.
num_tweets_to_process = 30
predicted_labels = []

for tweet in data['clean_tweet_text'][:num_tweets_to_process]:
    # Tokenize the tweet using Stanza (optional)
    doc = nlp(tweet)

    # Combine the tokens from all sentences into one string
    preprocessed_tweet = ' '.join([sent.text for sent in doc.sentences])

    input_ids = tokenizer.encode(preprocessed_tweet, add_special_tokens=True, truncation=True, max_length=512, padding='max_length', return_tensors='pt')

    with torch.no_grad():
      outputs = model(input_ids)
      logits = outputs.logits

      probabilities = torch.softmax(logits, dim=1)
      predicted_class = torch.argmax(probabilities, dim=1).item()
      sentiment_scores = probabilities[0].tolist()

#Using stanza for getting the sentences and sentiment
    for sent in doc.sentences:
      predicted_labels.append(sent.sentiment)
      print("Sentence :",sent.text)
      print( "Sentiment :",sent.sentiment)
#Using RobertA model for getting the sentiment scores
      print("Sentiment Scores:", sentiment_scores)
      print(" ")



In [ ]:
predicted_labels

In [ ]:
#For the whole tweets
predicted = []

for tweet in data['clean_tweet_text']:
    # Tokenize the tweet using Stanza (optional)
    doc = nlp(tweet)

    # Combine the tokens from all sentences into one string
    preprocessed_tweet = ' '.join([sent.text for sent in doc.sentences])

    input_ids = tokenizer.encode(preprocessed_tweet, add_special_tokens=True, truncation=True, max_length=512, padding='max_length', return_tensors='pt')

    with torch.no_grad():
      outputs = model(input_ids)
      logits = outputs.logits

      probabilities = torch.softmax(logits, dim=1)
      predicted_class = torch.argmax(probabilities, dim=1).item()
      sentiment_scores = probabilities[0].tolist()

    for sent in doc.sentences:
      predicted.append(sent.sentiment)
      print("Sentence :",sent.text)
      print( "Sentiment :",sent.sentiment)
      print("Sentiment Scores:", sentiment_scores)
      print(" ")

In [ ]:
#calculations of accuracy score
true_labels = train['label']
predicted_labels = predicted[:31962]
accuracy = accuracy_score(true_labels,predicted_labels)
print(accuracy)


In [ ]:
#Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Display the confusion matrix using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['negative', 'neutral', 'positive'], yticklabels=['negative', 'neutral', 'positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()